In [1]:
import random, math
import pandas as pd
import numpy as np
import os
import sklearn
from sklearn import datasets
import functools
from IPython.display import display, clear_output
random.seed(12345)

Para cada experimento se pasarán ciertos datos extra que serán datos que tendremos en cuenta al momento de experimentar, por ejemplo la página más referenciada y sus valores, o las principales que hacen que nosotros tengamos en cuenta ciertos principios a la hora de evaluar

In [2]:
# Generar txt y guardar instancia
def save_instanceA(dataset, instance_name, matriz): 
    if(not os.path.isdir(F"instancias/{dataset}")):
        os.mkdir(F"instancias/{dataset}")
                
    with open(F"instancias/{dataset}/{instance_name}.txt", "w") as f:
        for i in range(0, len(matriz)):
            for j in range(0, len(matriz[i])):
                print(matriz[i][j], file=f, end=" ")
            print(file=f, end="\n")
            
# Generar txt y guardar instancia
def save_instanceB(dataset, instance_name, vector): 
    if(not os.path.isdir(F"instancias/{dataset}")):
        os.mkdir(F"instancias/{dataset}")
                
    with open(F"instancias/{dataset}/{instance_name}.txt", "w") as f:
        for i in range(0, len(vector)):
            print(vector[i], file=f, end=" ")
        print(file=f, end="\n")

In [17]:
def guardar_caso_instancias_enCSV(folder_name, arr_instancias):
    pd.DataFrame(arr_instancias, columns=["dataset", "archivo-A","archivo-b", "n"]).to_csv(F"instancias/{folder_name}/indice.csv", index=False, header=True)
    
def guardar_caso_instancias_enCSV_SP(folder_name, arr_instancias):
    pd.DataFrame(arr_instancias, columns=["dataset", "archivo", "n", "type"]).to_csv(F"instancias/{folder_name}/indice.csv", index=False, header=True)

## Matrices triangulares - Multiples B

### Genero los "b"

In [5]:
def generate_b(n):
    b = np.random.rand(n)
    return b

In [6]:
def construir_matriz_completa(a,b,c):
    n = len(b)
    A = np.zeros((n,n))
    for i in range(0, len(b)):
        A[i][i] = b[i]
        if i >= 1:
            A[i][i-1] = a[i]
        if i < len(b)-1:
            A[i][i+1] = c[i]
            
    return A

### Matrices EDD

In [8]:
def tr_edd(n):
    a = np.random.rand(n)
    b = np.zeros(n)
    c = np.random.rand(n)
    for i in range(0, len(a)):
        sum_val = np.abs(a[i]) + np.abs(c[i])
        b[i] = sum_val*n
        
    a[0] = 0
    c[-1] = 0
    A = [a, b, c]
    return A

### Matrices SDP (Simetricas Definidas Positivas)

In [9]:
def tr_simetric_definite_positive(n):
    A = sklearn.datasets.make_spd_matrix(n)
    
    a = np.zeros(n)
    b = np.zeros(n)
    c = np.zeros(n)
    
    for i in range(0, len(A)):
        b[i] = A[i][i]
    
    for i in range(1, len(A)):
        a[i] = A[i][i-1]
    
    for i in range(0, len(A)-1):
        c[i] = A[i][i+1]
    
    B = [a, b, c]
    
    return B

### Matrices Random

In [10]:
def tr_random_matrix(n, nmin, nmax):

    A = np.random.randint(nmin, nmax, size=n)
    B = np.random.randint(nmin, nmax, size=n)
    C = np.random.randint(nmin, nmax, size=n)

    L = np.zeros((n, n))
    for i in range(0, n):
        L[i][i] = 1

    for i in range(1, n):
        L[i][i-1] = A[i-1]

    U = np.zeros((n, n))
    for i in range(0, n):
        U[i][i] = B[i]

    for i in range(0, n-1):
        U[i][i+1] = C[i]

    # SIN CUDA INSTALADO
    M = L @ U
    
    # CON CUDA INSTLADO
    # M = matmul_gpu(L, U)

    B = np.diagonal(M)
    
    for i in range(1, n):
        A[i] = M[i][i-1] 

    for i in range(0, n-1):
        C[i] = M[i][i+1]
    
    return [A, B, C]

## Matrices Tridiagonales - 4a

In [13]:
caso="tr_edd";
arr_instancias = []
for i in range(1, 31):
    clear_output(wait=True)
    display('Instancia: ' + str(i))
    n = 100*i
    A = tr_edd(n)
    b = generate_b(n)
    save_instanceA(caso, F"{caso}-{i}-A", A)
    save_instanceB(caso, F"{caso}-{i}-b", b)
    arr_instancias.append([caso, F"{caso}-{i}-A.txt", F"{caso}-{i}-b.txt", n])
guardar_caso_instancias_enCSV(caso,arr_instancias)

'Instancia: 30'

In [14]:
caso="tr_sdp";
arr_instancias = []
for i in range(1, 31):
    clear_output(wait=True)
    display('Instancia: ' + str(i))
    n = 100*i
    A = tr_simetric_definite_positive(n)
    b = generate_b(n)
    save_instanceA(caso, F"{caso}-{i}-A", A)
    save_instanceB(caso, F"{caso}-{i}-b", b)
    arr_instancias.append([caso, F"{caso}-{i}-A.txt", F"{caso}-{i}-b.txt", n])
guardar_caso_instancias_enCSV(caso,arr_instancias)

'Instancia: 30'

In [15]:
caso="tr_random";
arr_instancias = []
for i in range(1, 31):
    clear_output(wait=True)
    display('Instancia: ' + str(i))
    n = 100*i
    nmin = random.randint(1, 100)
    nmax = random.randint(101, 1000)
    A = tr_random_matrix(n, nmin, nmax)
    b = generate_b(n)
    save_instanceA(caso, F"{caso}-{i}-A", A)
    save_instanceB(caso, F"{caso}-{i}-b", b)
    arr_instancias.append([caso, F"{caso}-{i}-A.txt", F"{caso}-{i}-b.txt", n])
guardar_caso_instancias_enCSV(caso,arr_instancias)

'Instancia: 30'

## Matrices Tridiagonales - 4b

In [40]:
# Matrices EDD
caso="tr_edd_l";
arr_instancias = []
n = 5000
A = tr_edd(n)
save_instanceA(caso, F"{caso}-A", A)
for j in range(0, 1001):
    b = generate_b(n)
    save_instanceB(caso, F"{caso}-{j}-b", b)
    arr_instancias.append([caso, F"{caso}-{j}-b.txt", n, "b"])
arr_instancias.append([caso, F"{caso}-A.txt", n, "A"])
guardar_caso_instancias_enCSV_SP(caso, arr_instancias)

'Instancia: 2'

In [41]:
# Matrices SDP
caso="tr_sdp_l";
arr_instancias = []
n = 5000
A = tr_simetric_definite_positive(n)
save_instanceA(caso, F"{caso}-A", A)
for j in range(0, 1001):
    b = generate_b(n)
    save_instanceB(caso, F"{caso}-{j}-b", b)
    arr_instancias.append([caso, F"{caso}-{j}-b.txt", n, "b"])
arr_instancias.append([caso, F"{caso}-A.txt", n, "A"])
guardar_caso_instancias_enCSV_SP(caso, arr_instancias)

In [42]:
# Matrices Random
caso="tr_random_l";
arr_instancias = []
n = 5000
nmin = random.randint(1, 100)
nmax = random.randint(101, 1000)
A = tr_random_matrix(n, nmin, nmax)
save_instanceA(caso, F"{caso}-A", A)
for j in range(0, 1001):
    b = generate_b(n)
    save_instanceB(caso, F"{caso}-{j}-b", b)
    arr_instancias.append([caso, F"{caso}-{j}-b.txt", n, "b"])
arr_instancias.append([caso, F"{caso}-A.txt", n, "A"])
guardar_caso_instancias_enCSV_SP(caso, arr_instancias)